# Modelos de representación

## Librerías

In [2]:
# Librerias necesarias
import os

# Matplotlib conf
import matplotlib.pyplot as plt
%matplotlib inline
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':40,
         'ytick.labelsize': 40
}
plt.rcParams.update(params)
# Seaborn conf
import seaborn as sns
sns.set(style='darkgrid')
sns.set_palette(sns.color_palette("Blues"))

import sys

#Procesado de datos
import pandas as pd
import numpy as np
import operator

#Modelos
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')


from sklearn.decomposition import TruncatedSVD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn import preprocessing
from nltk import ngrams

import json

## Carga de los conjuntos de datos

Los conjuntos de datos que se utilizarán son los que se persistieron tras el preproceso:

In [3]:
df_training = pd.read_csv("./data/ds_training_processed.csv")
df_testing = pd.read_csv("./data/ds_testing_processed.csv")

X_train = df_training["text"]
y_train = df_training["sentiment"]
X_test = df_testing["text"]
y_test = df_testing["sentiment"]

y_train_2 = to_categorical(y_train, num_classes=2)
y_test_2 = to_categorical(y_test, num_classes=2)

## Conexión con el preprocesado

Dado que se pretende crear diversos modelos para analizarlos y luego realizar comparativas, será necesario utilizar diferentes modelos de representación. A pesar de esto, no resultaría lógico combinar algunos clasificadores con algunos modelos de representación. Pero, en cada uno de esos casos se justificará.

Con el propósito de crear estas representaciones, ya se adelantó en el preproceso la necesidad de reducir el vocabulario mediante las siguientes técnicas:
* Se eliminarán todas las palabras que solo aparezcan en 5 documentos.
* Se eliminarán todas las palabras que aparezcan en el 90% de los documentos.

Ciertamente, en el preproceso se planteó la reducción de vocabulario con el propósito de comprender mejor las tablas de frecuencias máximas. Pero, esta medida **también tiene otro propósito que es el de reducir la dimensionalidad de los modelos de representación**. Y esto se puede demostrar así:

In [15]:
base_cv = CountVectorizer(binary=True)
base_cv.fit(X_train)

vsm_binary_cv = CountVectorizer(binary=True, max_df=0.9, min_df=5)
vsm_binary_cv.fit(X_train)

display(
    base_cv.get_feature_names_out().shape,
    vsm_binary_cv.get_feature_names_out().shape
)

(27949,)

(8144,)

Se observa que la dimensionalidad del vocabulario del modelo de representación que ostenta las medidas adoptadas sobre las frecuencias de documento máximas y mínimas sobre el vocabulario, presenta una reducción de más de 4 veces menor que la base del experimento (sin las medidas).

Entonces, se ha demostrado una reducción de dimensionalidad mediante las medidas sobre las frecuencias adoptadas. De modo que en la construcción de los modelos de representación, ya se contará con este beneficio.

## Construcción de los modelos de representación en espacios vectoriales

Estos serán los modelos de representación que se crearán en espacios vectoriales:
* VSM + pesado binario + redimensionamiento por SVD (LSI).
* VSM + pesado binario + bigramas (ngrams) + redimensionamiento por SVD (LSI).
* VSM + pesado TF-IDF + redimensionamiento por SVD (LSI).
* VSM + pesado TF-IDF + bigramas (ngrams) + redimensionamiento por SVD (LSI).

### VSM + pesado binario + redimensionamiento por SVD (LSI)

In [16]:
vsm_binary_cv = CountVectorizer(binary=True, max_df=0.9, min_df=5)
vsm_binary_cv.fit(X_train)
svd = TruncatedSVD(n_components=5000)
vsm_binary = {
  "model": "VSM + pesado binario + redimensionamiento por SVD (LSI)",
  "train_text_features": svd.fit_transform( vsm_binary_cv.transform(X_train) ),
  "test_text_features": svd.transform( vsm_binary_cv.transform(X_test) )
}

### VSM + pesado binario + bigramas (ngrams) + redimensionamiento por SVD (LSI)

In [17]:
vsm_binary_ngrams_cv = CountVectorizer(binary=True, ngram_range=(1,2), max_df=0.9, min_df=5)
svd = TruncatedSVD(n_components=5000)
vsm_binary_ngrams = {
  "model": "VSM + pesado binario + rango de n-grams [1 y 2] + redimensionamiento por SVD (LSI)",
  "train_text_features": svd.fit_transform( vsm_binary_ngrams_cv.fit_transform(X_train) ),
  "test_text_features": svd.transform( vsm_binary_ngrams_cv.transform(X_test) )
}

### VSM + pesado TF-IDF + redimensionamiento por SVD (LSI)

In [18]:
vsm_tfidf_v = TfidfVectorizer(norm='l2', use_idf=True, max_df=0.9, min_df=5)
svd = TruncatedSVD(n_components=5000)
vsm_tfidf = {
  "model": "VSM + pesado TF-IDF + redimensionamiento por SVD (LSI)",
  "train_text_features": svd.fit_transform( vsm_tfidf_v.fit_transform(X_train) ),
  "test_text_features": svd.transform( vsm_tfidf_v.transform(X_test) )
}

### VSM + pesado TF-IDF + bigramas (ngrams) + redimensionamiento por SVD (LSI)

In [19]:
vsm_tfidf_ngrams_v = TfidfVectorizer(norm='l2', use_idf=True, ngram_range=(1,2), max_df=0.9, min_df=5)
svd = TruncatedSVD(n_components=5000)
vsm_tfidf_ngrams = {
  "model": "VSM + pesado TF-IDF + rango de n-grams [1 y 2] + redimensionamiento por SVD (LSI)",
  "train_text_features": svd.fit_transform( vsm_tfidf_ngrams_v.fit_transform(X_train) ),
  "test_text_features": svd.transform( vsm_tfidf_ngrams_v.transform(X_test) )
}

## Creación de modelos de representación con word embedding

Los modelos que se crearán son los siguientes:
* Word embedding: tokenizando el texto en secuencias y *pad_sequences*, para que tengan todos los textos de las observaciones el mismo tamaño.
* Word embedding con embeddings pre-entrenados (usando GloVe).

### Word embedding: tokenizando el texto en secuencias y *pad_sequences*

Primero se crea los tokens y las secuencias:

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1

print('Found %s unique tokens.' % len(tokenizer.word_index))

Found 29772 unique tokens.


Se han encontra 29.772 tokens diferentes en todo el conjunto de entrenamiento, lo cual define claramente el tamaño del vocabulario.

Por otra parte, ahora se dispone de secuencias de diferentes tamaños. Es decir: Un tamaño específico por cada documento. Por ejemplo, si el documento es de 2 palabras el tamaño de la secuencia será de 2 y si el documento es de 50 palabras, la secuencia será de 50.

Con el propósito de solventar el problema mencionado en el párrafo anterior, es necesario crear unificar todos los tamaños de las secuencias generadas. Para ello, se debe considerar cuál es el tamaño óptimo de la cadena:

In [16]:
max_len = 0
more_than_600 = 0
more_than_700 = 0
more_than_900 = 0
more_than_1000 = 0
for item in sequences_train:
  if len(item) > max_len:
    max_len = len(item)
  if len(item) > 600:
    more_than_600 = more_than_600 + 1
  if len(item) > 700:
    more_than_700 = more_than_700 + 1
  if len(item) > 900:
    more_than_900 = more_than_900 + 1
  if len(item) > 1000:
    more_than_1000 = more_than_1000 + 1

display(
    len(sequences_train),
    max_len
)
print("more_than_600:",more_than_600)
print("more_than_700:",more_than_700)
print("more_than_900:",more_than_900)
print("more_than_1000:",more_than_1000)

9993

1297

more_than_600: 26
more_than_700: 20
more_than_900: 8
more_than_1000: 6


Se observa que:
* La longitud máxima de secuencias es de 9.993. Es decir, que el mayor número de tokens que se encuentra en un texto es de 9.993.
* Por lo tanto, se podría estimar que el documento con mayor cantidad de tokens defina el tamaño de todas las secuencias. Aunque, **esta solución no sería adecuada porque produce más espacidad y dimensionalidad**.
* Por eso se ha contado cuántos documentos tienen un determinado número de tokens, contando a partir de 600 hasta 1000, considerando que 9.993 es un número que podría despreciarse. Sobre estos cortes, se puede observar que el número de documentos de más de 600 tokens es de 26. Y que este número se reduce poco a poco conforme se va aumentando el número de tokens.
* En conclusión, el número de documentos que tiene una longitud de 600 es apenas de 26, un número de documentos despreciable que supone apenas un *0.26%* del total de 9.993 documentos que se disponen.

Entonces, como ya se ha podido definir un número de secuencias para cortar y hacer padding en los casos que sea necesarios, se procede a hacer la operación y así crear finalmente el modelo de representación:

In [17]:
max_len = 600
embedding_padding = {
  "model": "Word embedding: tokenizando el texto en secuencias y pad_sequences",
  "train_text_features": pad_sequences(sequences_train, maxlen=max_len),
  "test_text_features": pad_sequences(sequences_test, maxlen=max_len)
}

### Word embedding con embeddings pre-entrenados (usando GloVe)

Para la creación de este modelo de representación:
* Se usará Glove entrenado con Wikipedia 2014 para generar representaciones de palabras en 50 dimensiones. Esto se descarga de:
    * https://nlp.stanford.edu/projects/glove/
    * Específicamente, se descarga el siguiente fichero: https://nlp.stanford.edu/data/glove.6B.zip
    * Se utilizará exactamente el fichero glove.6B.50d.txt
* Se fijará un tamaño de vocabulario de 50.000 palabras. Considerando que esta dimensionalidad tendrá un impacto en los tiempos de entrenamiento de los modelos clasificadores que se vayan a crear.

In [18]:
#Dimensión de la capa de embeddings
emb_dim = 50
#Número de palabras tenidas en cuenta (reducción del vocabulario)
n_most_common_words = 50000

embeddings_index = {}
f = open('./glove/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

num_words = min(n_most_common_words, len(tokenizer.word_index)) + 1
embedding_matrix = np.zeros((num_words, emb_dim))

for word, i in tokenizer.word_index.items():
    if i > n_most_common_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Si la palabra existe añadimos su vector a la matriz
        embedding_matrix[i] = embedding_vector
    else:
        # Si no existe, se le asigna un vector aleatorio
        embedding_matrix[i] = np.random.randn(emb_dim)

embedding_glove = {
  "model": "Word embedding with GloVe",
  "train_text_features": pad_sequences(sequences_train, maxlen=max_len),
  "test_text_features": pad_sequences(sequences_test, maxlen=max_len),
  "embedding_matrix": embedding_matrix
}

## Almacenamiento de los modelos de representación

Hay que considerar que el proceso de generar los modelos de representación se ve afectados por la reducción de dimensionalidad aplicada y que este proceso está tomando alrededor de 40 minutos. Así que, como los modelos de representación se utilizarán en diferentes fases, se deben de poder almacenar a los efectos de evitar repetir su proceso de creación, tal y como se haría con los modelos clasificadores.

Para ello se utilizará un sistema de persistencia basado en la persistencia de datos de *numpy*:

In [10]:
def save_representation_model(model_to_save, file_name):
    with open("./repmodels/"+file_name+".txt", 'w+') as f:
        f.write(model_to_save["model"])
    
    np.save("./repmodels/"+file_name+"_train.npy", model_to_save["train_text_features"])
    np.save("./repmodels/"+file_name+"_test.npy", model_to_save["test_text_features"])

    if "embedding_matrix" in model_to_save:
        np.save("./repmodels/"+file_name+"_embeddingmatrix.npy", model_to_save["embedding_matrix"])

def load_representation_model(file_name, include_embedding_matrix=False):
    res = {}
    with open("./repmodels/"+file_name+".txt", 'r') as d:
        res["model"] = d.readline()
    res["train_text_features"] = np.load("./repmodels/"+file_name+"_train.npy")
    res["test_text_features"] = np.load("./repmodels/"+file_name+"_test.npy")

    if include_embedding_matrix:
        res["embedding_matrix"] = np.load("./repmodels/"+file_name+"_embeddingmatrix.npy")

    return res

In [25]:
save_representation_model(vsm_binary, "vsm_binary")
save_representation_model(vsm_binary_ngrams, "vsm_binary_ngrams")
save_representation_model(vsm_tfidf, "vsm_tfidf")
save_representation_model(vsm_tfidf_ngrams, "vsm_tfidf_ngrams")
save_representation_model(embedding_padding, "embedding_padding")
save_representation_model(embedding_glove, "embedding_glove")